In [8]:
import csv
import os
from typing import Final
from pymonad.tools import curry
from pymonad.either import Left, Right, Either

In [28]:
# The IO Monad class
class IO:
    def __init__(self, action):
        self.action = action

    def execute_action(self):
        try:
            return self.action()
        except Exception as e:
            return Left(str(e))

    def bind(self, func):
        result = self.execute_action()
        if isinstance(result, Either):
            if result.is_left():
                return result
            else:
                try:
                    return func(result.value)
                except Exception as e:
                    return Left(str(e))
        else:
            try:
                return func(result)
            except Exception as e:
                return Left(str(e))

    def then(self, func):
        return self.bind(func)

In [30]:
# Function to handle file reading, now properly wrapped in IO monad
def read_csv_file(file_path):
    def read_file():
        if not os.path.exists(file_path):
            return Left("Error: File not found")
        with open(file_path, 'r') as csvfile:
            reader = csv.reader(csvfile)
            return Right([row for row in reader])
    return IO(read_file)

def remove_header(data):
    return (
        Right(data[1:]) 
        if len(data) > 1 
        else Left("Error: Unable to remove header")
    )

@curry(2)
def extract_column(column_index, data):
    return (
        Right(data).bind(lambda rows: 
        Right(list(map(lambda row: row[column_index], rows))))
    )

extract_score_column = extract_column(1)
extract_name_column = extract_column(0)

def convert_to_float(data):
    return (
        Right(list(map(float, data))) 
        if data 
        else Left("Error: Unable to convert to float")
    )


def calculate_average(column_values):
    return  (
        Right(sum(column_values) / len(column_values)) 
        if column_values 
        else Left("Error: Division by zero")
    )

In [31]:
# Data pipeline using the Either monad and custom sequencing operator
csv_file_path = 'example.csv'

In [32]:
data =  read_csv_file(csv_file_path)

In [48]:
data

In [43]:
names = (
    read_csv_file(csv_file_path)
    .then(remove_header)
    .then(extract_name_column)
)

result = (
    read_csv_file(csv_file_path)
    .then(remove_header)
    .then(extract_score_column)
    .then(convert_to_float)
    .then(calculate_average)
)

# Check if the results are Right and proceed accordingly
if result.is_right() and names.is_right():
    names_list = names.value
    average_score = result.value
    print(f"An average score of {', '.join(names_list[:-1])} and {names_list[-1]} is {average_score}")
else:
    print("Error processing data")

An average score of Alice, Bob, Charlie, David, Eva, Frank and Grace is 86.71428571428571


In [37]:
names_list

['Alice', 'Bob', 'Charlie', 'David', 'Eva', 'Frank', 'Grace']

In [39]:
names

Right ['Alice', 'Bob', 'Charlie', 'David', 'Eva', 'Frank', 'Grace']

In [45]:
result

Right 86.71428571428571

In [46]:
result.value

86.71428571428571